# 1. Get Source data for All _District Download_. web scraping

year='2017'
'https://rptsvr1.tea.texas.gov/perfreport/tapr/{}/xplore/DownloadSelData.html'.format(year)

step1
- 2021-22	https://rptsvr1.tea.texas.gov/perfreport/tapr/2022/xplore/DownloadSelData.html
- 2020-21	https://rptsvr1.tea.texas.gov/perfreport/tapr/2021/xplore/DownloadSelData.html
- 2018-19	https://rptsvr1.tea.texas.gov/perfreport/tapr/2019/xplore/DownloadSelData.html
- 2017-18	https://rptsvr1.tea.texas.gov/perfreport/tapr/2018/xplore/DownloadSelData.html
- 2016-17	https://rptsvr1.tea.texas.gov/perfreport/tapr/2017/xplore/DownloadSelData.html

step2 (**web scraping start from here**)

- 2021-22	https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&prgopt=2022/xplore/setdists.sas&year4=2022&_program=perfrept.perfmast.sas&sumlev=D&steps=2
- 2016-17	https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&prgopt=2017/xplore/setdists.sas&year4=2017&_program=perfrept.perfmast.sas&sumlev=D&steps=2

step3
- https://rptsvr1.tea.texas.gov/cgi/sas/broker

In [1]:
import pandas as pd
import numpy as np
import time
import re
import os
import pickle
import requests

from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
chrome_service = Service('chromedriver_mac_arm64/chromedriver')   # edit based on the version of used web browser 

years=[2017,2018,2019,2021,2022]
for year in years:
    locals()[f'me_{year}'] = pd.DataFrame(columns=['year','metric','element'])

    chrome_options = webdriver.ChromeOptions()
    prefs = {'download.default_directory' :  os.getcwd()+'/TX_data/TAPR_District/{}'.format(year)}; 
    chrome_options.add_experimental_option('prefs', prefs)
    #chrome_options.add_argument('--headless')   # uncomment if want to run in backend
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

    # Go to the first website
    driver.get("https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&prgopt={}/xplore/setdists.sas&year4={}&_program=perfrept.perfmast.sas&sumlev=D&steps=2".format(year,year))

    # Wait for the "continue" button to be clickable
    continue_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//input[@value='Continue']"))
    )
    continue_button.click()

    # Select the select element with name "dsname"
    dsname_select = Select(driver.find_element(by=By.NAME, value="dsname"))

    # Loop over all options
    m=[]
    for option in dsname_select.options:
        dsname_select.select_by_value(option.get_attribute("value"))
        m.append(option.get_attribute("value"))

    for option in m:
        option_element = driver.find_element(by=By.XPATH, value=f"//option[@value='{option}']")
        option_element.click()
        
        # Wait for the "continue" button to be clickable
        continue_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@value='Continue']"))
        )
        continue_button.click()
        
        # Choose excel format (There is no csv fomat from the first 3 years)
        # option_csv=driver.find_element(by=By.XPATH,value=f"//option[@value='X']")  # xls format. 
        # option_csv.click()

        # get all value of checkbox
        checkboxes = driver.find_elements(by=By.XPATH, value="//input[@type='checkbox']")

        data_element=[]
        for checkbox in checkboxes:
            data_element.append(checkbox.get_attribute("value"))
        
        # Wait for the "select all" button to be clickable
        select_all_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@value='Select All']"))
        )
        select_all_button.click()
        
        # Wait for the "download" button to be clickable &  Download
        download_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@value='Download']"))
        )
        download_button.click()

        locals()[f'me_{year}'] = pd.concat([locals()[f'me_{year}'], pd.DataFrame({'year': year, 'metric': option, 'element': data_element})], ignore_index=True)
        
        driver.back()
    
    # Make sure all downaload have been finished
    time.sleep(10) 
    driver.quit()
    
    locals()[f'me_{year}'] = locals()[f'me_{year}'].groupby(['year', 'metric'])['element'].apply(list).reset_index()

## Combine file into 1 dataframe according to its year.

In [ ]:
def merge_files(folder_path):
    all_files = os.listdir(folder_path)
    dataframes = []
    for file in all_files:
        file_path = os.path.join(folder_path, file)
        if file.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file.endswith('.xls'):
            df = pd.read_html(file_path)[0]
        else:
            print(file_path)
            continue
        dataframes.append(df)
    return pd.concat(dataframes, axis=0)


years=[2017,2018,2019,2021,2022]
for year in years:
    folder_path =  os.getcwd()+'/TX_data/TAPR_District/{}'.format(year)
    merged_df = merge_files(folder_path)
    merged_df.to_csv(os.path.join( os.getcwd(), f"TX_data/TAPR_District/merged_{year}.csv"), index=False)

# 2. Get all metric & elements

## 1) Get all metric
'https://rptsvr1.tea.texas.gov/perfreport/tapr/{}/xplore/taprref.html'.format(year)



In [ ]:
years=[2017,2018,2019,2021,2022]
for year in years:
    locals()[f'df_metric_{year}'] = pd.DataFrame(columns=['year','metric','meaning','href'])
    metric_url='https://rptsvr1.tea.texas.gov/perfreport/tapr/{}/xplore/taprref.html'.format(year)
    
    response = requests.get(metric_url)
    html = response.content
    soup = bs(html, "html.parser")

    for tag in soup.find_all('p'):
        try:
            context = tag.text.strip()
            if context.startswith('d'):
                link = tag.find('a').get('href')
                
                if link.endswith('html'):
                    link=metric_url.rstrip('taprref.html')+link
                
                metric=context.split('--')[0].strip()
                meaning=tag.text.split('--')[1].strip()
                
                #locals()[f'df_metric_{year}'] = locals()[f'df_metric_{year}'].append({'year':year,'metric':metric,'meaning':meaning,'href': link}, ignore_index=True)
                new_row = pd.DataFrame({'year':year,'metric':metric,'meaning':meaning,'href': link})
                locals()[f'df_metric_{year}'] = pd.concat([locals()[f'df_metric_{year}'], new_row], ignore_index=True)
        except:
            continue


### Check if they have same metric among 5 different years

In [459]:
df_concat = pd.concat([df_metric_2017,df_metric_2018,df_metric_2019,df_metric_2021,df_metric_2022])

# create a new column 'first_meaning' containing the first value of the 'meaning' column for each 'metric'
first_meanings = df_concat.groupby('metric').last()['meaning'].reset_index()

# join 'first_meaning'
metric_year = df_concat[['metric', 'year']].merge(first_meanings, on='metric', how='left')

# groupby 'metric' and aggregate 'year' into a list
metric_year= metric_year.groupby(['metric', 'meaning']).agg({'year': lambda x: x.tolist()})

# reset index and rename the columns
metric_year = metric_year.reset_index()
metric_year['metric']=metric_year['metric'].str.lower()
metric_year.rename(columns={'year': 'occurrence_year'}, inplace=True)
metric_year

,metric,meaning,occurrence_year
0,dadv,Advanced/Dual-Credit Course Completion (Grades...,"[2017, 2018, 2019, 2021, 2022]"
1,dapib,AP/IB Results,"[2017, 2018, 2019, 2021, 2022]"
2,dbil1,"BE/ESL: Approaches Grade Level, Meets Grade Le...","[2017, 2018, 2019, 2021, 2022]"
3,dbil2,BE/ESL Academic Growth: All Grades ELA/Reading...,"[2017, 2018, 2019, 2022]"
...,...,...,...
52,dstaar_ssi8,Student Success Initiative (Grade 8),"[2018, 2019]"
53,dstaf,Staff Information,"[2017, 2018, 2019, 2021, 2022]"
54,dstud,Student Information,"[2017, 2018, 2019, 2021, 2022]"
55,dtxihe,Texas Institution of Higher Education (TX IHE)...,"[2018, 2019, 2021, 2022]"


## 2) Get all dataelement inside each metric

In [ ]:
df3=pd.DataFrame(columns=['metric','elements'])
for index, row in metric_year.iterrows():
    metric = row['metric']
    years = row['occurrence_year']
    
    data_elements=[]
    for year in years:
        element_url='https://rptsvr1.tea.texas.gov/perfreport/tapr/{}/xplore/{}.html'.format(year,metric)
        html_content = requests.get(element_url)
        soup = bs(html_content.text, 'html.parser')

        # Find the table element in the HTML
        table = soup.find('table')
        for td in table.find_all('td')[::4]:
            element = td.text.strip()
            data_elements.append(element)
    data_elements=list(set(data_elements))
        
    new_row = pd.DataFrame({'metric': [metric], 'elements': [data_elements]})
    df3 = pd.concat([df3, new_row], ignore_index=True)

In [463]:
df3.head(5)

,metric,elements
0,dadv,"[DL9ADS18R, DR0ADE15R, DE9ADC18R, DH9ADE19R, D..."
1,dapib,"[DH0BTE18R, D20BTA15R, D20BKC15R, DA0BKA21R, D..."
2,dbil1,"[DDT00AS01S18R, DD500A001218R, DDJ00AR01319R, ..."
3,dbil2,"[DDA00AR01217R, DDT00AS01S18R, DQ00AM03A17R, D..."
4,dbil3,"[DJ408FR18R, DX00AR02G16R, DL00AR02E16R, DX00A..."


## Combine and rename variable above

In [470]:
years=[2017,2018,2019,2021,2022]

# Combine each year - metric -element.
for year in years:
    locals()[f'df_me_{year}'] = pd.merge(locals()[f'df_metric_{year}'],locals()[f'me_{year}']  , on=['year','metric'])

# Combine above
df_me=pd.merge(df3, metric_year,on='metric')

# Convert to upper case
def convert_column_to_uppercase(df, column_name):
    df[column_name] = df[column_name].str.upper()
    return df

df_list = [df_metric_2017,df_metric_2018,df_metric_2019,df_metric_2021,df_metric_2022,df_me]
for f in df_list:
    f = convert_column_to_uppercase(f, 'metric')

In [623]:
df_me

,metric,elements,meaning,occurrence_year
0,DADV,"[DL9ADS18R, DR0ADE15R, DE9ADC18R, DH9ADE19R, D...",Advanced/Dual-Credit Course Completion (Grades...,"[2017, 2018, 2019, 2021, 2022]"
1,DAPIB,"[DH0BTE18R, D20BTA15R, D20BKC15R, DA0BKA21R, D...",AP/IB Results,"[2017, 2018, 2019, 2021, 2022]"
2,DBIL1,"[DDT00AS01S18R, DD500A001218R, DDJ00AR01319R, ...","BE/ESL: Approaches Grade Level, Meets Grade Le...","[2017, 2018, 2019, 2021, 2022]"
3,DBIL2,"[DDA00AR01217R, DDT00AS01S18R, DQ00AM03A17R, D...",BE/ESL Academic Growth: All Grades ELA/Reading...,"[2017, 2018, 2019, 2022]"
...,...,...,...,...
52,DSTAAR_SSI8,"[DR8GPPR18R, DR08AMAP519R, DL08AMAF518R, DM8GP...",Student Success Initiative (Grade 8),"[2018, 2019]"
53,DSTAF,"[DPSOTOFC, DPSTVOFC, DPSTBIFC, DPSTSPFC, DPSTT...",Staff Information,"[2017, 2018, 2019, 2021, 2022]"
54,DSTUD,"[DE0GR16N, DPEMPCIP, DPETSPHP, DPEMWHIP, DPEMH...",Student Information,"[2017, 2018, 2019, 2021, 2022]"
55,DTXIHE,"[DHHEE20R, DW0GV19R, DAHEE20R, DWHEC19R, D40GV...",Texas Institution of Higher Education (TX IHE)...,"[2018, 2019, 2021, 2022]"


## Save above variables

In [476]:
# Save the variables to a file
with open("TX_data/variables.pickle", "wb") as f:
    pickle.dump((me_2017,me_2018,me_2019,me_2021,me_2022,
                 df_metric_2017,df_metric_2018,df_metric_2019,df_metric_2021,df_metric_2022,
                 df_me_2017,df_me_2018,df_me_2019,df_me_2021,df_me_2022,
                 metric_year,df3), f)  
    
    
with open("TX_data/variables2.pickle", "wb") as f:
    pickle.dump((df_me),f)

In [37]:
# # Load the variables from the file
# with open("TX_data/variables.pickle", "rb") as f:
#     me_2017,me_2018,me_2019,me_2021,me_2022,df_metric_2017,df_metric_2018,df_metric_2019,df_metric_2021,df_metric_2022,df_me_2017,df_me_2018,df_me_2019,df_me_2021,df_me_2022,metric_year,df3= pickle.load(f)

# with open("TX_data/variables2.pickle", "rb") as f:
#     df_me = pickle.load(f)

# 3. PDF to table
Link:
- 2021-22	https://rptsvr1.tea.texas.gov/perfreport/tapr/2022/datadict.pdf
- 2020-21	https://rptsvr1.tea.texas.gov/perfreport/tapr/2021/datadict.pdf
- 2018-19	https://rptsvr1.tea.texas.gov/perfreport/tapr/2019/datadict.pdf
- 2017-18	https://rptsvr1.tea.texas.gov/perfreport/tapr/2018/datadict.pdf
- 2016-17	https://rptsvr1.tea.texas.gov/perfreport/tapr/2017/datadict.pdf

Note: Above link also cover campus data. We only need to choose district table

In [ ]:
# Download datadict via its url
from urllib.request import urlretrieve
years=['2017','2018','2019','2021','2022']
for year in years:
    datadict_url='https://rptsvr1.tea.texas.gov/perfreport/tapr/{}/datadict.pdf'.format(year)
    urlretrieve(datadict_url, 'datadict/datadict_'+year+'.pdf')

https://rptsvr1.tea.texas.gov/perfreport/tapr/2017/datadict.pdf
https://rptsvr1.tea.texas.gov/perfreport/tapr/2018/datadict.pdf
https://rptsvr1.tea.texas.gov/perfreport/tapr/2019/datadict.pdf
https://rptsvr1.tea.texas.gov/perfreport/tapr/2021/datadict.pdf
https://rptsvr1.tea.texas.gov/perfreport/tapr/2022/datadict.pdf


## Extract table from pdf via Azure Form Recognizer
manually delete 'campus' part. Only leave district part.

In [480]:
from IPython.display import display
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import PyPDF2
import io

endpoint_pay='input_your_endpoint'
key_pay='input_your_key'

def datadict_csv_pay(endpoint,key):

    in_path = "TX_data/datadict/district/"
    allfiles = os.listdir(in_path)
    files = [ fname for fname in allfiles if fname.endswith('.pdf')]

    for filename in files:

        document_analysis_client = DocumentAnalysisClient(
            endpoint=endpoint_pay, credential=AzureKeyCredential(key_pay)
        )
        with open(in_path+filename, "rb") as f:
            poller = document_analysis_client.begin_analyze_document(
                "prebuilt-layout", document=f
            )
        result = poller.result()

        tables = []
        output=pd.DataFrame()
        for table_idx, table in enumerate(result.tables):
            df = [['' for i in range(table.column_count)] for j in range(table.row_count)] 

            for cell in table.cells:
                df[cell.row_index][cell.column_index] = cell.content         

            df=pd.DataFrame(df)
            output=pd.concat([output,df],axis=0,ignore_index=True)
            empty_row = pd.DataFrame({col: [None] for col in output.columns})
            output=pd.concat([output,empty_row],axis=0,ignore_index=True)
            
        output.to_csv(in_path+filename[:-4]+'.csv',header=False,index=False)
        

In [ ]:
## Use free key.
def datadict_csv(endpoint,key):
    in_path = "TX_data/datadict/district/"
    allfiles = os.listdir(in_path)
    files = [ fname for fname in allfiles if fname.endswith('.pdf')]

    for filename in files:
        
        # Open the PDF
        with open(in_path+filename, 'rb') as file:
            pdf = PyPDF2.PdfFileReader(file)
            
            # Get the number of pages
            num_pages = pdf.getNumPages()
            
            # Divide the number of pages by 2 to get the number of output PDFs
            num_output_pdfs = num_pages // 2
            
            # Loop over the output PDFs
            output=pd.DataFrame()
            for i in range(num_output_pdfs):
                # Create a new PDF
                output_pdf = PyPDF2.PdfFileWriter()
                
                # Add the first 2 pages to the new PDF
                output_pdf.addPage(pdf.getPage(i * 2))
                output_pdf.addPage(pdf.getPage(i * 2 + 1))
                
                document_analysis_client = DocumentAnalysisClient(
                    endpoint=endpoint, credential=AzureKeyCredential(key)
                )

                with io.BytesIO() as f:
                    output_pdf.write(f)
                    f.seek(0)
                    poller = document_analysis_client.begin_analyze_document(
                    "prebuilt-layout", document=f.read()
                        )
                    
                result = poller.result()

                tables = []
                
                for table_idx, table in enumerate(result.tables):

                    print("Table # {} has {} rows and {} columns; Table Page # is {}".format(table_idx, table.row_count, table.column_count,table.bounding_regions[0].page_number))    

                    df = [['' for i in range(table.column_count)] for j in range(table.row_count)] 
                    for cell in table.cells:
                        df[cell.row_index][cell.column_index] = cell.content         

                    df=pd.DataFrame(df)
                    output=pd.concat([output,df],axis=0,ignore_index=True)
                    empty_row = pd.DataFrame({col: [None] for col in output.columns})
                    output=pd.concat([output,empty_row],axis=0,ignore_index=True)
                
            output.to_csv(in_path+filename[:-4]+'.csv',header=False,index=False)
        

# 4. Datadict process.
process datadict file from 2017-2022

In [ ]:
def datadict_process(df2):

    empty_rows = df2.loc[df2.iloc[:,1:].isna().all(axis=1)].index.tolist()
    df2 = df2.applymap(str)
    df2 = df2.replace('\n:unselected:','', regex=True)
    df2 = df2.replace(':unselected:', '', regex=True)
    df2 = df2.replace('nan', '', regex=True)

    # use ffill (forward fill) to fill empty strings in new column with values from first column
    df2.insert(0, 'Data field', '')
    for i in range(len(empty_rows)-1):
        start_index = empty_rows[i]
        end_index = empty_rows[i+1]
        df2.iloc[start_index:end_index, 0] = df2.iloc[start_index, 1]

    df2=df2.rename(columns={'Unnamed: 0':'Subcategory'})
    df2.loc[:,'Subcategory'] = df2.loc[:,'Subcategory'].replace('', method='ffill')

    return df2

input_path =  os.getcwd()+'/TX_data/datadict/district/'
output_path =  os.getcwd()+'/TX_data/datadict/district_processed/'
csv_files = [os.path.join(input_path, f) for f in os.listdir(input_path) if f.endswith(".csv") and f.startswith('datadict')]

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df = datadict_process(df)
    
    base_name = os.path.splitext(csv_file)[0].replace(input_path,'')
    new_name = base_name+ ".csv"  #  + suffix 
    df.to_csv(output_path + new_name, index=False)

# 5. Table Mapping


In [6]:
with open("TX_data/variables2.pickle", "rb") as f:
    df_me = pickle.load(f)

In [ ]:
"""
Organize the meanings of abbreviations
"""

# abb=pd.read_csv('TX_data/abbreviations_explanation.csv')
# df_split = abb['meaning'].str.split('(', expand=True)
# abb['meaning']=df_split[0]
# abb['abbr'] = abb['abbr'].combine_first(df_split[1].str.replace(')', '') )
# abb.to_csv('TX_data/abbreviations.csv')

In [18]:
"""
Input one element, 
it will find all releated element in the past five years
return a dataframe with all metric and its district

Note: State and Region ## column not applicate.
"""

abb=pd.read_csv('TX_data/abbreviations.csv')

def get_row_number(input, df):
    row_numbers = []
    for i, row in df.iterrows():
        if row['position_index']=='0':
             if input[0] == row['abbr']:
                row_numbers.append(i)
        elif row['position_index']=='1':
            if input[1] == row['abbr']:
                row_numbers.append(i)
        else:
            if input[1:3] == row['abbr']:
                row_numbers.append(i)        
    return df.iloc[row_numbers,2:]
    #get_row_number(input, abb)

def select_columns(df, columns):
    existing_columns = set(df.columns)
    selected_columns = [col for col in columns if col in existing_columns]
    return df[selected_columns]

def mapping5(enum):
    try:
        position = df_me[df_me['elements'].apply(lambda x: enum in x)].index.tolist()[0]
        e_years=df_me.loc[position,'occurrence_year']
        e_metric=df_me.loc[position,'metric']
        meaning=df_me.loc[position,'meaning']
        print(enum,' belongs to metric: ',e_metric) 
        print('Occurrence year: ', e_years,)
        print("Meaning: ",meaning)
        
        display(get_row_number(enum, abb))

        ae=[]
        for ind,i in enumerate(e_years):
            ae.append(enum[:-3] + str(i)[2:4] + enum[-1])
            ae.append(enum[:-3] + str(i-1)[2:4] + enum[-1])
            ae.append(enum[:-3] + str(i-2)[2:4] + enum[-1])
        ae=list(set(ae))
        ae.sort()
        ae.insert(0,'DISTRICT')

        for ind,i in enumerate(e_years):
            
            if i<=2019:
                df = pd.read_html(os.path.join('TAPR_District',str(i), e_metric+'.xls'))[0]
            else:
                df = pd.read_csv(os.path.join('TAPR_District',str(i), e_metric+'.csv'))
            
            dc= select_columns(df, ae)
            
            if ind==0:
                doutput=dc
            else:
                doutput=pd.merge(doutput,dc,on='DISTRICT',how='outer')
        
        doutput=doutput.T.drop_duplicates().T
        return doutput
    
    except:
        print('please make sure ', enum,' not belong to State and Region ## column')

In [19]:
df41=mapping5('DH00AS03A17R') 
df41

DH00AS03A17R  belongs to metric:  DSTAAR6
Occurrence year:  [2017]
Meaning:  District STAAR Data - Masters Grade Level (All Grades)


,position_index,meaning,abbr,type1,type2
2,0,district,D,NaN,NaN
39,1,Hispanic,H,"Student Groups Available for Postsecondary, At...",Student Groups


,DISTRICT,DH00AS03A16R,DH00AS03A17R
0,'001902,-1,-1
1,'001903,-1,-1
2,'001904,-1,-1
3,'001906,-1,-1
4,'001907,8,15
...,...,...,...
1198,'252902,-1,-1
1199,'252903,-1,-1
1200,'253901,6,7
1201,'254901,5,11


## Input a list of elements

In [16]:
"""
Input element list
it will find all releated elements in the past five years and meaning of each element
return a dataframe with all metric and its district
"""

def mapping5_list(enum_list):
    path_list=[]
    ae=[] 
    for enum in enum_list:
        try:
            position = df_me[df_me['elements'].apply(lambda x: enum in x)].index.tolist()[0]
        except:
            print('please make sure ', enum,' not belong to State and Region ## column')
            continue
            
        e_years=df_me.loc[position,'occurrence_year']
        e_metric=df_me.loc[position,'metric']
        print('occurrence_year: ', e_years)
        print('metric: ',e_metric)

        for ind,i in enumerate(e_years):
            ae.append(enum[:-3] + str(i)[2:4] + enum[-1])
            ae.append(enum[:-3] + str(i-1)[2:4] + enum[-1])
            ae.append(enum[:-3] + str(i-2)[2:4] + enum[-1])
            path_list.append(os.path.join('TAPR_District',str(i), e_metric))
            
    ae=list(set(ae))
    ae.sort()
    ae.insert(0,'DISTRICT')
    path_list=list(set(path_list))
    path_list.sort()
        
    for ind,path in enumerate(path_list):
        print(path)
        if int(path.split('/')[1])<=2019:
            df = pd.read_html(path+'.xls')[0]
        else:
            df = pd.read_csv(path+'.csv')
        dc= select_columns(df, ae)
        
        if ind==0:
            output=dc
        else:
            output=pd.merge(output,dc,on='DISTRICT',how='outer')
    
    # Deal with duplicate columns. Save the one occur in later year.
    output = output[output.columns[~output.columns.str.endswith('_x')]]
    output=output.rename(columns={col: col.rstrip('_y') for col in output.columns if col.endswith('_y')})
    output.sort_index(axis=1, ascending=False, inplace=True)
    
    # Sort columns by its name
    cols = output.columns.tolist()
    cols.sort()
    cols.remove('DISTRICT')
    cols.insert(0, 'DISTRICT')
    output = output[cols]
    return output

In [625]:
enum_list= ['DDA03ARE1219R','DDA03ARE1218R','DB0GR18N']
d42=mapping5_list(enum_list)
d42

occurrence_year:  [2018, 2019, 2021, 2022]
metric:  DSTAAR_GR3
occurrence_year:  [2018, 2019, 2021, 2022]
metric:  DSTAAR_GR3
occurrence_year:  [2018, 2019, 2021, 2022]
metric:  DGRAD
TAPR_District/2018/DGRAD
TAPR_District/2018/DSTAAR_GR3
TAPR_District/2019/DGRAD
TAPR_District/2019/DSTAAR_GR3
TAPR_District/2021/DGRAD
TAPR_District/2021/DSTAAR_GR3
TAPR_District/2022/DGRAD
TAPR_District/2022/DSTAAR_GR3


,DISTRICT,DB0GR17N,DB0GR18N,DB0GR20N,DB0GR21N,DDA03ARE1217R,DDA03ARE1218R,DDA03ARE1219R,DDA03ARE1221R,DDA03ARE1222R
0,'001902,3,1,2,2,42,57,61,71,74
1,'001903,6,2,6,5,50,55,35,55,58
2,'001904,5,5,2,3,42,54,43,59,53
3,'001906,1,3,3,1,57,55,30,61,65
...,...,...,...,...,...,...,...,...,...,...
1214,'015842,NaN,NaN,NaN,.,NaN,NaN,NaN,NaN,.
1215,'015843,NaN,NaN,NaN,.,NaN,NaN,NaN,NaN,.
1216,'101877,NaN,NaN,NaN,.,NaN,NaN,NaN,NaN,.
1217,'152504,NaN,NaN,NaN,.,NaN,NaN,NaN,NaN,71
